In [1]:
import random
import json
from datetime import datetime

# Constants
COLLEGES = ["College of Engineering", "College of Science", "College of Arts", 
            "College of Medicine", "College of Business", "College of Law"]
DEPARTMENTS = [f"Department {i+1}" for i in range(28)]
VENUE_CAPACITIES = list(range(50, 321, 10))  # 50 to 320 in steps of 10
VENUES = [f"Venue {i+1}" for i in range(30)]
TIME_SLOTS = [f"{hour}:00" for hour in range(8, 13)] + ["14:00", "15:00", "16:00", "17:00"]
LUNCH_BREAK = "13:00"
DATES = [f"2024-01-{day:02d}" for day in range(8, 36) if (day - 8) % 7 != 6]  # 4 weeks, Mon-Sat
INVIGILATORS = [f"Invigilator {i+1}" for i in range(20)]

def generate_data():
    data = {
        "colleges": [],
        "departments": [],
        "courses": [],
        "students_summary": [],
        "venues": [],
        "invigilators": [],
        "time_slots": TIME_SLOTS,
        "lunch_break": LUNCH_BREAK,
        "dates": DATES,
        "metadata": {
            "version": "v1.0",
            "generated_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    }

    # Colleges and Departments
    for college in COLLEGES:
        college_departments = random.sample(DEPARTMENTS, 4)
        data["colleges"].append({"name": college, "departments": college_departments})
        data["departments"].extend(college_departments)
    
    # Courses
    for department in data["departments"]:
        for level in range(100, 500, 100):  # Level 100 to 400
            num_courses = random.randint(10, 14)
            courses = [f"{department} Course {level}-{i+1}" for i in range(num_courses)]
            data["courses"].extend([{"name": course, "level": level, "department": department} for course in courses])

    # Students Summary
    for department in data["departments"]:
        students_per_level = {
            "level_100": random.randint(1, 200),
            "level_200": random.randint(20, 150),
            "level_300": random.randint(20, 120),
            "level_400": random.randint(20, 100)
        }
        data["students_summary"].append({"department": department, "students": students_per_level})

    # Venues
    for venue, capacity in zip(VENUES, VENUE_CAPACITIES):
        data["venues"].append({"name": venue, "capacity": capacity})

    # Invigilators
    data["invigilators"] = [{"name": inv} for inv in INVIGILATORS]

    return data

# Save JSON file
with open("data7.json", "w") as f:
    json.dump(generate_data(), f, indent=4)

print("Data generated and saved to data7.json")


Data generated and saved to data7.json


In [40]:
import json
import random
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

# Constants
LUNCH_BREAK = "13:00"
DEFAULT_COLUMN_WIDTH = 20

# Load generated data
with open("data7.json", "r") as f:
    data = json.load(f)

# Helper functions
def get_available_time_slots():
    """Return all time slots excluding lunch break."""
    # Add start and end times for each slot
    time_slots = {
        "8:00": "8:00 AM - 10:00 AM",
        "10:30": "10:30 AM - 12:30 PM",
        "14:00": "2:00 PM - 4:00 PM",
        "16:30": "4:30 PM - 6:30 PM",
    }
    return {slot: label for slot, label in time_slots.items() if slot != LUNCH_BREAK}

def assign_invigilators(invigilators, num_needed=2):
    """Round-robin assign invigilators."""
    invigilator_pool = iter(invigilators)
    assigned = []
    for _ in range(num_needed):
        try:
            assigned.append(next(invigilator_pool))
        except StopIteration:
            invigilator_pool = iter(invigilators)
            assigned.append(next(invigilator_pool))
    return assigned

# Main Timetable Generation Logic
def generate_timetable(data):
    """
    Generate the exam timetable based on constraints.
    """
    timetable = []
    unscheduled_courses = []
    available_time_slots = get_available_time_slots()
    max_exams_per_day = len(data["venues"]) * len(available_time_slots)  # Total slots available per day
    total_courses = len(data["courses"])

    # Calculate how many days are needed
    required_days = (total_courses + max_exams_per_day - 1) // max_exams_per_day  # Ceiling division
    print(f"Total courses to schedule: {total_courses}")
    print(f"Max exams per day: {max_exams_per_day}")
    print(f"Days available: {len(data['dates'])}")
    print(f"Days needed: {required_days}")

    current_day_index = 0
    venue_usage = defaultdict(lambda: defaultdict(list))  # Track venue usage by day and time slot

    # Process courses
    for course in data["courses"]:
        department = course["department"]
        level = course["level"]
        course_name = course["name"]

        # Get student count
        student_count = next(
            (summary["students"].get(f"level_{level}", 0) for summary in data["students_summary"] if summary["department"] == department),
            0
        )

        # Allocate a venue
        venue = None
        assigned_time_slot = None

        while current_day_index < len(data["dates"]):  # Ensure we don't exceed available days
            current_day = data["dates"][current_day_index]
            day_of_week = datetime.strptime(current_day, "%Y-%m-%d").strftime("%A")
            for time_slot, time_range in available_time_slots.items():
                # Find an available venue for the time slot
                venue = next(
                    (v for v in data["venues"] if v["name"] not in venue_usage[current_day][time_slot] and v["capacity"] >= student_count),
                    None
                )
                if venue:
                    assigned_time_slot = time_range
                    # Mark venue as used
                    venue_usage[current_day][time_slot].append(venue["name"])
                    break

            if assigned_time_slot:
                break
            else:
                # Move to the next day if no time slot is available
                current_day_index += 1

        # If no day or venue is left, mark the course as unscheduled
        if not assigned_time_slot or not venue:
            print(f"DEBUG: No more days or venues available to schedule {course_name}.")
            unscheduled_courses.append({"course": course_name, "reason": "No available slots or venues", "students": student_count})
            continue

        # Assign invigilators
        assigned_invigilators = assign_invigilators(data["invigilators"], num_needed=2)

        # Add to timetable
        timetable.append({
            "Day": day_of_week,
            "Date": current_day,
            "Time": assigned_time_slot,
            "Course": course_name,
            "Department": department,
            "Level": level,
            "Venue": venue["name"],
            "Invigilators": ", ".join(inv["name"] for inv in assigned_invigilators),
            "Exam_type": "Written" if level >= 300 else "CBT"  # CBT for Level 100/200, Written for Level 300/400
        })

    print(f"Total exams successfully scheduled: {len(timetable)}")
    print(f"Total unscheduled courses: {len(unscheduled_courses)}")
    return timetable, unscheduled_courses

# Save Timetable and Unscheduled Courses to Excel
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill

def save_timetable_to_excel(timetable, unscheduled_courses, file_name="timetable.xlsx"):
    """
    Save the timetable and unscheduled courses to an Excel file and highlight column headers in yellow.
    """
    with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
        # Save timetable
        df_timetable = pd.DataFrame(timetable)
        df_timetable.to_excel(writer, index=False, sheet_name="Exam Timetable")

        # Adjust column widths and highlight header
        workbook = writer.book
        worksheet = writer.sheets["Exam Timetable"]

        yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        for col_idx, col_name in enumerate(df_timetable.columns, start=1):
            column_letter = get_column_letter(col_idx)
            worksheet[f"{column_letter}1"].fill = yellow_fill  # Apply yellow fill to header cell

            # Adjust column width
            max_length = max(len(str(value)) for value in df_timetable[col_name].values if pd.notnull(value))
            max_length = max(max_length, len(col_name))  # Include header length
            worksheet.column_dimensions[column_letter].width = max_length + 2  # Add padding

        # Save unscheduled courses
        if unscheduled_courses:
            df_unscheduled = pd.DataFrame(unscheduled_courses)
            df_unscheduled.to_excel(writer, index=False, sheet_name="Unscheduled Courses")

            # Adjust column widths and highlight header
            worksheet_unscheduled = writer.sheets["Unscheduled Courses"]
            for col_idx, col_name in enumerate(df_unscheduled.columns, start=1):
                column_letter = get_column_letter(col_idx)
                worksheet_unscheduled[f"{column_letter}1"].fill = yellow_fill  # Apply yellow fill to header cell

                # Adjust column width
                max_length = max(len(str(value)) for value in df_unscheduled[col_name].values if pd.notnull(value))
                max_length = max(max_length, len(col_name))
                worksheet_unscheduled.column_dimensions[column_letter].width = max_length + 2

    print(f"Timetable successfully saved to {file_name}")



# Generate and Save Timetable
try:
    timetable, unscheduled_courses = generate_timetable(data)
    save_timetable_to_excel(timetable, unscheduled_courses)
except Exception as e:
    print(f"Error generating timetable: {e}")


Total courses to schedule: 1174
Max exams per day: 112
Days available: 24
Days needed: 11
Total exams successfully scheduled: 1174
Total unscheduled courses: 0
Timetable successfully saved to timetable.xlsx
